In [142]:
%matplotlib inline

import os
import numpy as np
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm


import matplotlib.pyplot as plt

try:
    from hgtk.text import compose, decompose
except:
    !pip install hgtk
    from hgtk.text import compose, decompose

## 시 데이터 셋 :  백석 시 10편
----

이번 시간에는 "RNN을 활용한 시 작문하기"를 실습해보도록 하겠습니다.<br>
시는 백석 시인의 시 10편을 선별하여 학습하고, <br>
모델이 백석 시인의 작품을 생성할 수 있는지 확인해보도록 하겠습니다.

In [2]:
if not os.path.exists("./poets.zip"):
    !wget https://pai-datasets.s3.ap-northeast-2.amazonaws.com/alai-deeplearning/poets.zip
if not os.path.exists("./poets"):
    !unzip poets.zip
    
def read_poets(poet_dir):
    """시가 담겨져 있는 디렉토리 내에서 시 내용들을 가져오는 함수
    :param poet_dir : *.txt 포맷으로 저장된 시가 담긴 디렉토리 위치
    
    :return 
        list of the poet(type: str)
    """
    poets = []
    for poet_name in os.listdir(poet_dir):
        poet_path = os.path.join(poet_dir, poet_name)
        with open(poet_path,'r') as f:
            poets.append(f.read())
    return poets

# 백석 시를 읽어옮
poets = read_poets("poets/")
print("총 백석 시의 편 수 : ",len(poets))

print(poets[0])

총 백석 시의 편 수 :  10
가난한 내가
아름다운 나타샤를 사랑해서 
오늘밤은 푹푹 눈이 나린다 
나타샤를 사랑은 하고 
눈은 푹푹 날리고 
나는 혼자 쓸쓸히 앉어 소주를 마신다 
소주를 마시며 생각한다 
나타샤와 나는 눈이 푹푹 쌓이는 밤
흰 당나귀 타고 산골로 가자 출출이 우는
깊은 산골로 가 마가리에 살자 
눈은 푹푹 나리고 
나는 나타샤를 생각하고 
나타샤가 아니올 리 없다 
언제 벌써 내 속에 고조곤히 와 이야기한다 
산골로 가는 것은 세상한테 지는 것이 아니다 
세상 같은 건 더러워 버리는 것이다 
눈은 푹푹 나리고 
아름다운 나타샤는 나를 사랑하고 
어데서 흰 당나귀도 오늘밤이 좋아서 응앙응앙 울을 것이다 


<br>

# \[ 시를 작문하는 RNN 모델 구성하기 \]
----
----

> *백석 시를 학습하고, 시 문구를 넣었을 때, 백석 시와 비슷한 시를 생성해내는 모델을 만들어 보도록 하겠습니다.<br>

<br>

## 1. Word Embedding 하기
---
---

* 우리는 한글 자모자를 기준으로 임베딩을 하도록 하겠습니다.<Br>

### (1) 한글 자모자 구성하기

In [3]:
# 한글 자모자를 인덱스로 만드는 Map 구현
초성 = (
    u'ㄱ', u'ㄲ', u'ㄴ', u'ㄷ', u'ㄸ', u'ㄹ', u'ㅁ', u'ㅂ', u'ㅃ', u'ㅅ',
    u'ㅆ', u'ㅇ', u'ㅈ', u'ㅉ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
)

중성 = (
    u'ㅏ', u'ㅐ', u'ㅑ', u'ㅒ', u'ㅓ', u'ㅔ', u'ㅕ', u'ㅖ', u'ㅗ', u'ㅘ',
    u'ㅙ', u'ㅚ', u'ㅛ', u'ㅜ', u'ㅝ', u'ㅞ', u'ㅟ', u'ㅠ', u'ㅡ', u'ㅢ', u'ㅣ'
)

종성 = (
    u'', u'ㄱ', u'ㄲ', u'ㄳ', u'ㄴ', u'ㄵ', u'ㄶ', u'ㄷ', u'ㄹ', u'ㄺ',
    u'ㄻ', u'ㄼ', u'ㄽ', u'ㄾ', u'ㄿ', u'ㅀ', u'ㅁ', u'ㅂ', u'ㅄ', u'ㅅ',
    u'ㅆ', u'ㅇ', u'ㅈ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
)

미포함종성 = tuple(set(종성) - set(초성)) 

### (2) 임베딩 dictionary 구현하기


| 자모자 | 인덱스 |
| ---- | --- |
|ㄱ| 0 |
|ㄲ| 1 |
|ㄴ| 2 |
|ㄷ| 3 |
|ㄸ| 4 |
|ㄹ| 5 |
|...| ... |

와 같은 순으로 매칭시키도록 하겠습니다.

In [4]:
# 초성, 중성, 종성, 그리고 "ᴥ"를 포함
jamos = list(초성 + 중성 + 미포함종성 + ("ᴥ"," ","\n",)) 
# jamo에 매칭되는 인덱스
jamo2idx = { jamo : idx for idx, jamo in enumerate(jamos) }

<br>

## 2. Language Modeling
---
---

* **언어 모델(language model)** 은 문장, 즉 단어 나열에 확률을 부여하고, 예측하는 모델을 말합니다. 특정한 단어의 시퀀스에 대해, 그렇게 단어가 배치될 가능성이 어느 정도인지(얼마나 자연스러운 단어 순서인지)를 확률로 평가하는 것입니다.
* 예를 들어, "나 밥 먹으러 가"라는 단어 시퀀스에는 높은 확률(예:0.092)를 출력하고, "나 비행기 먹으러 가"라는 시퀀스에는 낮은 확률(예:0.00001)을 출력하는 것이 일종의 언어 모델입니다.

### (1) 문장 확률(동시확률) 정의

하나의 단어(word)를 w라고 합시다. 단어의 시퀀스인 전체 문장(sentence)를 대문자 W라고 합시다. n개의 w로 구성된 문장 W의 확률은 다음과 같이 표현할 수 있습니다.

$
P(W) = p(w_1,w_2,w_3,\cdots,w_n) = \prod_{i=1}^n p(w_i)
$

### (2) 조건부 언어 모델(conditional language model)

우리는 이미 앞서 등장한 단어를 바탕으로 다음 단어를 예측할 수 있습니다. 예를 들어, n-1개의 단어가 나열된 상태에서 n번째 단어의 확률을 다음과 같이 표현할 수 있습니다.<br>
$
P(w_n|w_1,w_2,w_3,\cdots,w_{n-1})
$

예를 들어 다섯번째 단어의 확률은 아래처럼 표기할 수 있습니다.<br>

$
P(w_5|w_1,w_2,w_3,w_4)
$

위에서 구한 문장의 확률은 위의 조건부 확률로 분해해서 바라볼 수 있습니다.<br>

$
P(w_1,\cdots,w_n) = P(w_n|w_1,\cdots,w_{n-1})P(w_{n-1}|w_1,\cdots,w_{n-2})\\
\cdots P(w_3|w_1,w_2)P(w_2|w_1)P(w_1) \\
= \prod_{i=1}^{n}p(w_t|w_1,\cdots,w_{i=1})
$


### (3) 언어 모델의 직관적 해석

````
KTX를 타러 서울역에 도착했는데, 짐 싸다 늦어서 KTX를 [?]
````

라는 문장이 있다고 생각해봅시다. 우리는 앞서 나열된 단어들을 통해 직관적으로 `[?]`에 들어갈 내용은 "놓쳤다"임을 예상할 수 있습니다. 우리는 사전의 나열된 정보를 바탕으로 사후적으로 다음 단어를 판단할 수 있기 때문입니다. <br>
언어 모델도 동일하게 단어의 시퀀스를 통해, 다음 단어가 무엇인지 나올지를 판단하도록 학습함으로써, 문장의 맥락(Context)를 파악하는 방법을 학습하게 됩니다.

<br>

## 3. Poet Generator 구현하기
---
---

* 우리는 이전 시간에 다룬 삼성 Stock을 예측하는 모델과 같이 Stacked LSTM을 이용해 보도록 하겠습니다.
* 30개의 자모자를 읽고, 다음 자모자를 예측하는 Language Model을 학습시키도록 하겠습니다.

### (1) 단어를 벡터로 만들기

In [6]:
time_steps = 30 # 몇개의 자모자를 보고 다음 자모자를 예측할지 결정
word_ndims = len(jamos) # 임베딩한 자모자의 갯수

data = []
targets = []
for poet in poets:
    jamo_poet = decompose(poet) # 시의 음절을 자모자로 분리
     # 벡터화한 후 stack
    jamo_vectors = np.stack([jamo2idx[char] 
                             for char in jamo_poet 
                             if char in jamo2idx])

    len_seqs = len(jamo_vectors)
    for i in range(len_seqs-time_steps):
        # 하나씩 지나가면서, 데이터를 확보
        datum = jamo_vectors[i:i+time_steps]
        target = jamo_vectors[i+1:i+time_steps+1]
        data.append(datum)
        targets.append(target)
        
X = np.array(data)
Y = np.array(targets)

merged = list(zip(X,Y))

np.random.shuffle(merged)

X, Y = list(zip(*merged))
X = np.stack(X)
Y = np.stack(Y)

In [7]:
X.shape # 총 14704의 문장, 30개로 이루어진 time step

(14704, 30)

In [8]:
Y.shape # 우리가 예측하려는 문장도 같다

(14704, 30)

<br>

## 4. RNN 모델 만들기
---
---

* 우리는 이전 시간에 다룬 삼성 Stock을 예측하는 모델과 같이 Stacked LSTM을 이용해 보도록 하겠습니다.
* 30개의 자모자를 읽고, 다음 자모자를 예측하는 Language Model을 학습시키도록 하겠습니다.

### (1) Layer 구성하기

이번 경우, Inference 때와 Training 때 약간 다르게 동작합니다.<br>
이를 위해, 우리는 `keras.layers`에서의 Layer을 우선 선언한 후, 두 모델을 만들어야 합니다.

In [145]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Input, Dense, Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow.keras.backend as K

In [147]:
K.clear_session()

_, n_steps = X.shape
n_inputs = np.max(X) + 1

n_embedding = 16
n_units = 200

# 우선 Keras Layer를 구성
embedding_layer = Embedding(n_inputs, n_embedding) # 총 55개의 자모자(n_inputs)를 16차원(n_embedding)으로 줄입니다.
lstm1_layer = LSTM(units=n_units, 
                   return_sequences=True, 
                   return_state=True)
lstm2_layer = LSTM(units=n_units, 
                   return_sequences=True,
                   return_state=True)
lstm3_layer = LSTM(units=n_units,
                   return_sequences=True,
                   return_state=True)
dense_layer = Dense(n_inputs, activation='softmax')

### (2) Training Model 구성하기

아래와 같이 구성할 수 있습니다.

In [151]:
inputs = Input(shape=(None,), name='inputs')

embeded = embedding_layer(inputs)
hidden1, _, _ = lstm1_layer(embeded)
hidden2, _, _ = lstm2_layer(hidden1)
hidden3, _, _ = lstm3_layer(hidden2)
output = dense_layer(hidden3)

model = Model(inputs, output, name='training')

model.summary()

Model: "training"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 16)          880       
_________________________________________________________________
lstm (LSTM)                  [(None, None, 200), (None 173600    
_________________________________________________________________
lstm_1 (LSTM)                [(None, None, 200), (None 320800    
_________________________________________________________________
lstm_2 (LSTM)                [(None, None, 200), (None 320800    
_________________________________________________________________
dense (Dense)                (None, None, 55)          11055     
Total params: 827,135
Trainable params: 827,135
Non-trainable params: 0
____________________________________________________

### (2) Inference Model 구성하기

Training Model과 Inference Model은 같은 Weight를 공유하고 있는데, 차이는 Inference Model에서는 state 정보를 주입받아, 계산된 state를 반환합니다.

In [153]:
# Inference Model을 구성

## Initial State을 주입할 수 있도록 구성
h1_in = Input(shape=(n_units,), name='h1_input')
c1_in = Input(shape=(n_units,), name='c1_input')
h2_in = Input(shape=(n_units,), name='h2_input')
c2_in = Input(shape=(n_units,), name='c2_input')
h3_in = Input(shape=(n_units,), name='h3_input')
c3_in = Input(shape=(n_units,), name='c3_input')

in_states_1 = [h1_in, c1_in]
in_states_2 = [h2_in, c2_in]
in_states_3 = [h3_in, c3_in]
in_states = in_states_1 + in_states_2 + in_states_3

# 모델 구성하기
embedding_layer = Embedding(n_inputs, n_embedding)
hidden1, h1_out, c1_out = lstm1_layer(embeded,
                                      initial_state=in_states_1)
hidden2, h2_out, c2_out = lstm2_layer(hidden1,
                                      initial_state=in_states_2)
hidden3, h3_out, c3_out = lstm3_layer(hidden2,
                                      initial_state=in_states_3)
output = dense_layer(hidden3)

## output state를 받을 수 있도록 구성
out_states_1 = [h1_out, c1_out]
out_states_2 = [h2_out, c2_out]
out_states_3 = [h3_out, c3_out]
out_states = out_states_1 + out_states_2 + out_states_3

inference_model = Model([inputs]+in_states, 
                        [output]+out_states_1+out_states_2+out_states_3,
                        name='inference')
inference_model.summary()

Model: "inference"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 16)     880         inputs[0][0]                     
__________________________________________________________________________________________________
h1_input (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
c1_input (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________

### (3) Poet Generate 메소드 구현하기

우리는 첫 소절을 입력받아, 다음 글자를 생성하는 메소드를 구현하도록 하겠습니다.

In [154]:
def generate_poet(inference_model, test_string, generate_length=200):
    """ 주어진 첫 소절을 바탕으로, 이어서 쓰는 시
    """
    jamo_seqs = decompose(test_input_string)
    code_seqs = np.stack([jamo2idx[char] for char in jamo_seqs])
    initial_state = [np.zeros((1,200)) for _ in range(6)]

    outputs = inference_model.predict(
        [code_seqs[np.newaxis]]+initial_state)
    logits, states = outputs[0], outputs[1:]
    last_logits = logits[:,-1:,:] # 마지막 출력값은 다음 입력값이 됨
    last_word = np.argmax(last_logits, axis=-1)
    
    generated_sequence = [last_word]
    for _ in range(generate_length):
        # 입력값 + states 값을 Input으로 넣어줌
        outputs = inference_model.predict(
            [last_word] + states)
        logits, states = outputs[0], outputs[1:]
        
        # 마지막 출력값은 다음 입력값이 됨
        last_logits = logits[:,-1:,:]
        last_word = np.argmax(last_logits, axis=-1)
        generated_sequence.append(last_word)

    generated_sequence = np.stack(generated_sequence).ravel()    
    generated_poet = compose("".join([jamos[code]
                                      for code in generated_sequence]))
    return generated_poet

In [156]:
test_input_string = "내 어머니와 아버지는"
print(f"test input string >>> {test_input_string}")
print(generate_poet(inference_model, test_input_string))

test input string >>> 내 어머니와 아버지는
ㅒㅒㅒㅒㅒㅒㅒㅎㅎㅎㅎㅎㅎㅎㅎㅎㅍㅍㅍㅍㅍ퍠ㅒㅒㅒㄷㄷㄷㄷㄷㄷㄷ됴ㅛㅆㅆㅆㅆ쏘ㅗㅗㅗㅗㅋㅋㅅㅅㅅㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌㅌㅌㅌㅌㅌㅌㅅㅅㅅㅅㅅㅌㅌㅌㅌ


학습이 아직 되지 않았기 때문에, 이상하게 나타납니다. 학습함에 따라 점점 글자를 

### (4) 모델 학습시키기

In [138]:
# 출력 라벨이 자모자의 인덱스로 나오기 때문에 우리는 sparse_categorical_crossentropy
# 를 이용해야 합니다.
model.compile(Adam(lr=1e-3),
              loss=sparse_categorical_crossentropy)

epochs=10
test_input_string = "내 어머니와 아버지는"

for _ in range(epochs):
    model.fit(x=X,y=Y, batch_size=20)
    
    # 평가
    print("test : {} >".format(test_input_string))
    print(generate_poet(inference_model, test_input_string))
    
    # 데이터 셋의 순서를 섞어줌
    dataset = list(zip(X, Y))
    np.random.shuffle(dataset)
    X, Y = list(zip(*dataset))
    X,Y = np.stack(X), np.stack(Y)    

14704/14704 [==============================] - 117s 8ms/sample - loss: 0.9663
test : 흰 당나귀 타고 산골로 가자 출출이 우는 >


것이다
이것은 오는 것이다
이것은 오는 것이다
이것은 오는 것이다
이것은 오는 것이다
이것은 오는 것이다
이것은 오는 것이다
이
14704/14704 [==============================] - 121s 8ms/sample - loss: 0.7244
test : 흰 당나귀 타고 산골로 가자 출출이 우는 >


것이었다
그러나 줌시 다하와 하긋한 아븜우로 가는 것이 있다
내 가난한 늙은 어머니가
이렇게 서녁 간 당다는 만 옛적 큰 아바지가 
14704/14704 [==============================] - 116s 8ms/sample - loss: 0.5626
test : 흰 당나귀 타고 산골로 가자 출출이 우는 >


것이었다
내 가슴이 없는 것이다
이것은 아득한 옛날 한가하고 조이틀은 아이들끼리 앗간 한 방을 잡고 조아질하고 쌈방이 굴리고 
14704/14704 [==============================] - 118s 8ms/sample - loss: 0.4753
test : 흰 당나귀 타고 산골로 가자 출출이 우는 >


깊은 산골로 가자 출출이 우는
깊은 산골로 가자 출출이 우는
깊은 산골로 가자 출출이 우는
깊은 산골로 가자 출출이 우는
깊은
 1440/14704 [=>............................] - ETA: 1:54 - loss: 0.4345

KeyboardInterrupt: 

## reference : 
1. [language model](https://wikidocs.net/21668)